# Fantasy Analysis for Week 8

In [266]:
from sleeper_wrapper import League
from sleeper_wrapper import Players
import statistics
import plotly.graph_objects as go
import plotly.express as px
import requests
from plotly.subplots import make_subplots

LEAGUE_ID = "649912836461539328"
CURRENT_WEEK = 9
FORMAT = "half_ppr"
FANTASY_POSITIONS = ["QB", "RB", "WR", "TE", "FLEX", "DEF", "K"]
league = League(LEAGUE_ID)
sleeper_players = Players()

users = league.get_users()
rosters = league.get_rosters()

def pair_user_rosters(rosters):
    """pairs a users roster id with their user id"""
    pairings = {}

    for roster in rosters:
        pairings[roster["roster_id"]] = roster["owner_id"]

    return pairings


def pair_usernames(users):
    pairings = {}

    # this is stored dumb, but basically team name can be in metadata or it can
    # be outside of it. If a team has no name, we can just use their display name
    for user in users:
        try:
            pairings[user["user_id"]] = user["metadata"]["team_name"]
        except KeyError:  # stored dumb
            try:
                pairings[user["user_id"]] = user["team_name"]
            except KeyError:  # no team name
                pairings[user["user_id"]] = user["display_name"]

    return pairings


ROSTER_PAIRINGS = pair_user_rosters(rosters)
NAME_PAIRINGS = pair_usernames(users)
all_games = {}


def pair_matchups(games):
    """pair matchup data with users and their opponent"""
    scoreboards = {}

    for player in games:
        if player["matchup_id"] in scoreboards.keys():
            scoreboards[player["matchup_id"]
                        ][ROSTER_PAIRINGS[player["roster_id"]]] = player
        else:
            scoreboards[player["matchup_id"]] = {
                ROSTER_PAIRINGS[player["roster_id"]]: player}

    return scoreboards


# creates a dictionary with all matchups in the league through CURRENT_WEEK
for week in range(1, CURRENT_WEEK+1):
    all_games[week] = pair_matchups(league.get_matchups(week))


In [267]:
# how does median points scored differ from points per game?
# lets define breakout as 30+ points
BREAKOUT_CUT = 30
#   league discussion has been on points for (as an aside, how about opp. median score?)
# ADD STATS STUFF TO THIS FUNCTION BUT KEEP THIS UP TO DATE
STATS_FIELDS = ["Weeks", "Weekly Points For", "Weekly Points Against", "Points For", "Points Against", "Points per Game",
                "Median Points Scored", "Points Against per Game", "Median Points Against", "3 game rolling average",
                "5 game rolling average", "Breakout Games", "Breakout Games Against", "Game Margin", "Blowout Index"]

# blowout index: closer to 0 is close games, bigger numbers indicate larger losses and wins

#   TODO: edge case for less than 3 or 5 weeks?


def team_stats(games):
    """calculate team stats. returns a dictionary of team ids with associated
       lists that define team stats. list has order:
       [total weeks, weekly points for, weekly points against, total points for, total points against, ppg, 
       median points scored, papg, median points against, 3 game rolling avg. points scored, 
       5 game rolling avg. points scored, breakouts for, breakouts against, game margin, blowout index]
    """
    weeks = len(games.keys())
    final_stats = {}
    weekly_pf = {}
    weekly_pa = {}
    weekly_bf = {}
    weekly_ba = {}

    for user in NAME_PAIRINGS:
        final_stats[user] = []
        weekly_pf[user] = []
        weekly_pa[user] = []
        weekly_bf[user] = []
        weekly_ba[user] = []

    for week in games:
        for matchup in games[week]:
            teams = list(games[week][matchup].keys())
            team0 = games[week][matchup][teams[0]]
            team1 = games[week][matchup][teams[1]]
            # use team data to fill in pf and opponent data for pa
            weekly_pf[teams[0]].append(team0["points"])
            weekly_pf[teams[1]].append(team1["points"])
            weekly_pa[teams[0]].append(team1["points"])
            weekly_pa[teams[1]].append(team0["points"])
            weekly_bf[teams[0]].append({})
            weekly_bf[teams[1]].append({})
            weekly_ba[teams[0]].append({})
            weekly_ba[teams[1]].append({})
            for player0, player1 in zip(team0["players_points"].keys(), team1["players_points"].keys()):
                if team0["players_points"][player0] >= BREAKOUT_CUT:
                    weekly_bf[teams[0]][week -
                                        1][player0] = team0["players_points"][player0]
                    weekly_ba[teams[1]][week -
                                        1][player0] = team0["players_points"][player0]
                elif team1["players_points"][player1] >= BREAKOUT_CUT:
                    weekly_ba[teams[0]][week -
                                        1][player1] = team1["players_points"][player1]
                    weekly_bf[teams[1]][week -
                                        1][player1] = team1["players_points"][player1]

    for user in NAME_PAIRINGS:
        final_stats[user].append(weeks)
        final_stats[user].append(weekly_pf[user])
        final_stats[user].append(weekly_pa[user])
        pf = sum(weekly_pf[user])
        pa = sum(weekly_pa[user])
        final_stats[user].append(round(pf, 2))
        final_stats[user].append(round(pa, 2))
        final_stats[user].append(round(pf/weeks, 2))
        final_stats[user].append(round(statistics.median(weekly_pf[user]), 2))
        final_stats[user].append(round(pa/weeks, 2))
        final_stats[user].append(round(statistics.median(weekly_pa[user]), 2))

        rolling_3 = []
        for i in range(weeks):
            if i < 3:
                rolling_3.append(round(sum(weekly_pf[user][0:i+1])/(i+1), 2))
            else:
                rolling_3.append(round(sum(weekly_pf[user][i-2:i+1])/3, 2))
        final_stats[user].append(rolling_3)

        rolling_5 = []
        for i in range(weeks):
            if i < 5:
                rolling_5.append(round(sum(weekly_pf[user][0:i+1])/(i+1), 2))
            else:
                rolling_5.append(round(sum(weekly_pf[user][i-4:i+1])/5, 2))
        final_stats[user].append(rolling_5)

        final_stats[user].append([len(k.keys()) for k in weekly_bf[user]])
        final_stats[user].append([len(k.keys()) for k in weekly_ba[user]])
        final_stats[user].append(
            [round(weekly_pf[user][week] - weekly_pa[user][week], 2) for week in range(weeks)])
        final_stats[user].append(round(sum(
            [final_stats[user][-1][i]**2 for i in range(weeks)]) / weeks, 2))  # mean squared margin

        # ADD HERE

    return final_stats


stats = team_stats(all_games)

In [268]:
# sanity check the weeks and uncomment here if you wanna

# for week in all_games:
#     print(f"------------Week {week}------------")
#     for matchup in all_games[week]:
#         for team in all_games[week][matchup]:
#             print(f" {NAME_PAIRINGS[str(team)]}: {all_games[week][matchup][team]['points']}")
#         print()

## League Status

Hello boys, I enjoyed last week so I will be doing some more analysis, this time with regards to roster construction. As a reminder, league standings are currently as follows:

```
Noe's Loyal Lighting Salesman Division
1. Nahome (11-5)
2. Ben (8-8)
3. Noe (8-8)
4. Will (7-9)
5. Stephane (3-13)

Noah's SideThiccs
1. Noah (11-5)
2. Praveen (11-5)
3. Stian (9-7)
4. Tom (7-9)
5. Tegran (5-11)
```

This weeks power rankings that I released on Tuesday had the most movement in weeks,  as Nahome fell to third and Stephane's trade paid off in the week's second highest scoring performance.

```
1. Noah
2. Praveen
3. Nahome
4. Ben
5. Stian
6. Noe
7. Tom
8. Will
9. Stephane
10. Tegran
```

If the playoffs started today, we would have ```Praven vs. Noe``` and ```Stian vs. Ben``` in the opening rounds, with Noah and Nahome both earning byes. In the toilet bowl, we would have ```Tegran vs. Will``` and ```Stephane vs. Tom```.

## Weekly Focus

For the week 8 deep dive, I will be creating and digging into a stat that Tom inspired me to try as a joke. Tom asked to do a wins above replacement stat (WAR) for Aiyuk, who obviously has not been the best fantasy performer this year.  Diligent readers probably noticed I wrote 'Aiyuk is ASS' as his WAR value. Regardless, I wanted to do something similar for this week since it will be pretty interesting to see the degree to which a player impacts a team. In fantasy, some position groups are simply more important because the dropoffs in player output is so much more drastic (This is why we generally draft RBs before WRs before QBs!). Since we have less games to sample from, I will try to analyze points above replacement (PAR) for players on a roster. Hopefully this will give some insight into the top performers on a roster and their overall contributions to a players' team. 

## Applying Rigor to PAR (and PAA)
#### This section dives more deeply into the logic behind my calculations and motivation for creating two classes of statistic. I would recommend reading it, but if that doesn't interest you as much you can skip it

The definition of WAR as stated on the Baseball-Reference website is “A single number that presents the number of wins the player added to the team above what a replacement player . . . would add.” For their purposes, a replacement player is defined as a player taken from the top minor league level. Of course, we don't have this selection luxury in fantasy which begs the question of what our replacement calculation should use. There is also debate in the baseball community, which I think informs our decision making process. Just due to the nature of rosters and player ownership, an average player may actually be difficult to acquire and require a team to trade value in pursuing their acquisition. With that said, it is clear that a true 'replacement' level player is one that is able to be picked up just off the waiver wire. In this sense, most players we start on a weekly basis have a positive points above replacement statistic. This implies that our aggregate team PAR score might be judging our teams relative to one we can create off the waiver wire.

This then suggests that the average rostered player would give us a better sense of the amount a team is helped by an individual **relative** to other teams. Teams are generally not starting replacement players on a weekly basis, so does it really make sense to peg point production to players that are not started? Because of this fact, I came to the solution of binning players into tiers and comparing their point production to these tiers. In our league, we start 1 QB, 2 RBs, 2 WRs, 1 TE, and 2 FLEX players (as well as 1 team defense and 1 kicker) and have 10 total teams. If we can calculate what a player in a given starting spot is expected to score, we could analyze how many points they contribute relative to a starting lineup consisting solely of bang average players. Let's drill down into running backs as a concrete example of this methodology. If in our league the top 10 running backs in a week scored 20, 19, 18, 17, 16, 15, 14, 13, 12, and 11 points respectively, we might say that the expected point return of eachs team's top running back (their RB1) was 14.5 points. In this respect, we could infer that a replacement (or perhaps more specifically an average) RB1 would be worth 14.5 points to their team. Thus the top scoring RB would have added 5.5 points to their team. We will call this value the Points Added Over Average (PAA)

I believe all these statistics have uses. We saw just this week that Derrick Henry was hurt and Noah was forced to replace him off the waiver wire. We may expect his points to drop by the value of Derrick Henry's PAR score. We all know that Stephane has had a brutal start to his season. In his case, it may be more interesting to compare how the running backs he drafted are performing relative to the top RB options on other squads. For this case we can analyze the PAA stat of his starting RBs to infer the amount they have hurt his season compared to other positional players. On the opposite end of the spectrum, Nahome's star WR options will likely improve his aggregate PAA score since he has the luxury of playing top wideouts in his FLEX spots.

The presence of bye weeks presents a slight difficulty in these calculations. I am making the conscious decision to ignore bye weeks even though some players have not yet had them while others have (and thus have had an extra week of production). My logic is for injured players, I would not want to remove weeks for when they are injured from this calculation since they would have scored 0 points if you had started them. Since each player will have a bye week at some point this season, I think the logic for leaving bye week production as 0 points is even more sound than for including injury weeks in the calculation. Essentially the argument boils down to this: If you are relying on a player to be your RB1 and they score 0 points on their bye week, other teams are still starting players who are RB1s. I believe this difference in points scored should therefore still count towards a players' production output.

## Generating Player PAR Estimates

Generating PAR estimates will generally be simpler than PAA estimates. I will just use the average points scored by players on the waiver wire (who have scored a non-zero amount of points, under the assumption that this is the true pool of replacement players) to generate an estimate of the average score for a replacement player at a given position. To be upfront about this, I tried this originally but felt like the stat skewed a little low. I limited the average to the top 10 waiver wire pickups since those are the players that would be most likely to actually replace a rostered player. I will then compare this to the actual points scored by players on active rosters to generate the difference in points scored by a given player vs. this replacement pool. This stat can then be aggregated for a given team or for a position subgroup on a team to calculate the team and positional strength of a team relative to a team made up solely of replacement level players.

In [269]:
# lets grab all the players Sleeper has, lets only run this once cause the api says it does not need to be called more than once per day
players = sleeper_players.get_all_players()

In [270]:
# unfortunately, this just gives us the player metadata and not their stats
# the api wrapper im using doesn't enable us to grab stats directly, so we
# will do it ourselves with the requests package and the 
# api route: https://api.sleeper.app/stats/nfl/player/{XXXX}?season_type=regular&season=2021&grouping=season
# what's nice is that if a player has no stats, they just return a null response so we can populate our
# stats dictionary with only players that have a point very easily

# to update player stats, run stat_loading.py
# we load data from a file for performance reasons
# querying the API

import json

with open('fantasy_features/player_stats.json') as fp:
    player_stats = json.load(fp)


In [271]:
# since we already know from our api requests that players which return a response have stats, we just need 
# to filter on the pts_half_ppr field (which individual defensive players do not have)
scoring_offensive_players = {}

for player in player_stats:
    if "pts_half_ppr" in player_stats[player] and players[player]["position"] in FANTASY_POSITIONS:
        try:
            scoring_offensive_players[player] = {"points": player_stats[player]["pts_half_ppr"]}
            scoring_offensive_players[player]["fantasy_positions"] = players[player]["fantasy_positions"]
            scoring_offensive_players[player]["position"] = players[player]["position"]
            scoring_offensive_players[player]["name"] = players[player]["full_name"]
        except:
            scoring_offensive_players[player] = {"points": player_stats[player]["pts_half_ppr"]}
            scoring_offensive_players[player]["fantasy_positions"] = players[player]["fantasy_positions"]
            scoring_offensive_players[player]["position"] = players[player]["position"]
            scoring_offensive_players[player]["name"] = players[player]["player_id"]

In [272]:
rostered_players = {}

for roster in rosters:
    team = roster["players"]
    for player in team:
        # sometimes players on IR will have 0 points scored, so we can build their record if
        # they are rostered
        try:
            rostered_players = {**rostered_players, **{player: scoring_offensive_players[player]}}
        except KeyError:
            rostered_players = {**rostered_players, **{player: {"points": 0, 
                                                       "name": players[player]["full_name"], 
                                                       "position": players[player]["position"], 
                                                       "fantasy_positions": players[player]["fantasy_positions"]}}
                                                       }
to_remove = []
for player in scoring_offensive_players:
    if player in rostered_players:
        to_remove.append(player)

replacement_players = {key: scoring_offensive_players[key] for key in scoring_offensive_players if key not in rostered_players}

In [273]:
# we should now calculate the replacement average points for each position
# if limit is set to None, there is no limit on the replacement players and
# all are used to calculate the average. It may be more realistic to just use
# the top # = LIMIT players on the waiver wire at a position, since those are
# the replacement options you would actually pickup
replacement_averages = {}
LIMIT = 10

for position in FANTASY_POSITIONS:
    pos_sum = 0
    pos_players = 0 if not LIMIT else LIMIT
    top_players = []

    for player in replacement_players:
        # Flex encompasses three positional replacement possibilities, so we
        # must take all three into account
        if position == "FLEX":
            for subposition in ["RB", "WR", "TE"]:
                if subposition in replacement_players[player]["fantasy_positions"]:
                    if not LIMIT:
                        pos_sum += replacement_players[player]["points"]
                        pos_players += 1
                    else:
                        if len(top_players) < LIMIT:
                            top_players.append(
                                replacement_players[player]["points"])
                        else:
                            p_points = replacement_players[player]["points"]
                            top_players = [p_points if p_points > min(
                                top_players) else curr_top for curr_top in top_players]
                        pos_sum = sum(top_players)
        else:
            if position in replacement_players[player]["fantasy_positions"]:
                if not LIMIT:
                    pos_sum += replacement_players[player]["points"]
                    pos_players += 1
                else:
                    if len(top_players) < LIMIT:
                        top_players.append(
                            replacement_players[player]["points"])
                    else:
                        p_points = replacement_players[player]["points"]
                        top_players = [p_points if p_points > min(top_players) else curr_top for curr_top in top_players]
                    pos_sum = sum(top_players)

    
    replacement_averages[position] = pos_sum / pos_players

fig = go.Figure()

weekly_replacements = [val / CURRENT_WEEK for val in replacement_averages.values()]
fig.add_trace(
    go.Bar(x=list(replacement_averages.keys()), y=weekly_replacements,
           marker=dict(color=weekly_replacements, coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Replacement Player Weekly Score",
                  showlegend=False,
                  xaxis={'categoryorder': 'total descending'},
                  )

fig.show()


In [274]:
(sum(replacement_averages.values()) + replacement_averages["RB"] + replacement_averages["WR"] + replacement_averages["FLEX"]) / CURRENT_WEEK

90.65499999999997

We see above that we have generated a weekly score for a typical replacement level player at each position. If we were to construct a starting lineup from these players, it would be expected to score about 92 points. If we hover over the list, we see that a replacement quarteback is worth about 17 points on average, while a replacement tight end is worth about 6. FLEX players have a higher expected score than the position groups that make up the category because they have the benefit of choosing the highest performing players at three different positions. Certainly though I did not expect Kickers to have a higher replacement value than running backs or tight ends. It just goes to show how important those draft picks are to team success (looking at you Stephane).

Now that we have those values calculated for replacements, lets move onto the interesting part and calculate the difference between these values and a players' actual roster.

In [275]:
# After we have the expected replacement score, we can simply calculate the difference
# between this average score and a players' total score to find their points above replacement.

for roster in rosters:
    roster["PAR_values"] = []
    roster["PAR_values_weekly"] = []
    roster["PAR_names"] = []
    for player in roster["players"]:
        player_position = rostered_players[player]["position"]
        player_points = rostered_players[player]["points"]
        rostered_players[player]["PAR"] = player_points - replacement_averages[player_position]
        rostered_players[player]["PAR_weekly"] = (player_points - replacement_averages[player_position])/CURRENT_WEEK
        roster["PAR_values"].append(player_points - replacement_averages[player_position])
        roster["PAR_values_weekly"].append((player_points - replacement_averages[player_position])/CURRENT_WEEK)
        roster["PAR_names"].append(rostered_players[player]["name"])

# use this to sanity check with ben's roster
# for player, par_value in zip(rosters[0]["players"], rosters[0]["PAR_values"]):
#     print(f"{rostered_players[player]['name']} has scored {rostered_players[player]['points']} pts. This is {round(par_value, 2)} points {'above' if par_value > 0 else 'below'} a typical replacement {rostered_players[player]['position']}")


In [276]:
fig = make_subplots(rows=5, cols=2, shared_yaxes=True,
                    subplot_titles=list(NAME_PAIRINGS.values()))

for i in range(5):
    for j in range(1, 3):
        current = list(NAME_PAIRINGS.keys())[(i+i+j)-1]
        # hacky cause rosters is out of order, fix later if you want
        for roster in rosters:
            if roster["owner_id"] == current:
                current_roster = roster
    
        fig.add_trace(
            go.Bar(y=current_roster["PAR_values_weekly"],
                   x=current_roster["PAR_names"],
                   marker=dict(color=current_roster["PAR_values_weekly"],
                               cmid=0,
                               coloraxis="coloraxis")),
            i+1, j
        )

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Points Above Replacement per Week",
                  showlegend=False,
                  height= 2000, width=1500
                  )

fig.show()


Above, we can see the points above replacement per week for each player that is currently on a roster. There are some real standouts like Cooper Kupp and Derrick Henry. Most of the hugely negative players are ones who have been hurt most of the season like Jarvis Landry or Jerry Jeudy. If you hover over a bar, you can see the value for their points above replacement. Since I want to also use this as a measure of team strength, the next few charts will plot out some of the standout players and some of the rostered duds. Hopefully this will show how these guys get broken out into tiers and give some insight into the true value of certain players.

In [277]:
rostered_replacement_stars = {}
rostered_replacement_duds = {}

for position in FANTASY_POSITIONS:
    top_players = []
    bot_players = []
    
    for player in rostered_players:
        if position == "FLEX":
            continue

        else:
            if position in rostered_players[player]["fantasy_positions"]:
                if len(top_players) < LIMIT:
                    top_players.append(
                        rostered_players[player])
                else:
                    p_points = rostered_players[player]["PAR"]
                    pars = [p["PAR"] for p in top_players]
                    idx = pars.index(min(pars))
                    if p_points > pars[idx]:
                        top_players[idx] = rostered_players[player]

                if len(bot_players) < LIMIT:
                    bot_players.append(
                        rostered_players[player]
                    )
                else:
                    p_points = rostered_players[player]["PAR"]
                    pars = [p["PAR"] for p in bot_players]
                    idx = pars.index(max(pars))
                    if p_points < pars[idx]:
                        bot_players[idx] = rostered_players[player]
        
    rostered_replacement_stars[position] = top_players
    rostered_replacement_duds[position] = bot_players

names = [
    "QB Over Performers", "QB Under Performers", 
    "RB Over Performers", "RB Under Performers",
    "WR Over Performers", "WR Under Performers", 
    "TE Over Performers", "TE Under Performers",
    "Kickers", "Defenses"
]

fig = make_subplots(rows=5, cols=2, shared_yaxes=True,
                    subplot_titles=names)

for i in range(5):
    current = list(rostered_replacement_stars.keys())[i]
    star_par = [player["PAR"] / CURRENT_WEEK for player in rostered_replacement_stars[current]]
    star_name = [player["name"] for player in rostered_replacement_stars[current]]
    dud_par = [player["PAR"] / CURRENT_WEEK for player in rostered_replacement_duds[current]]
    dud_name = [player["name"] for player in rostered_replacement_duds[current]]

    fig.add_trace(
        go.Bar(y=star_par,
                x=star_name,
                marker=dict(color=star_par,
                            cmid=0,
                            coloraxis="coloraxis")),
        i+1, 1
    )

    fig.add_trace(
        go.Bar(y=dud_par,
                x=dud_name,
                marker=dict(color=dud_par,
                            cmid=0,
                            coloraxis="coloraxis")),
        i+1, 2
    )

k_par = [player["PAR"] /
            CURRENT_WEEK for player in rostered_replacement_stars["K"]]
k_name = [player["name"]
                for player in rostered_replacement_stars["K"]]
def_par = [player["PAR"] /
            CURRENT_WEEK for player in rostered_replacement_duds["DEF"]]
def_name = [player["name"]
            for player in rostered_replacement_duds["DEF"]]

fig.add_trace(
    go.Bar(y=k_par,
            x=k_name,
            marker=dict(color=k_par,
                        cmid=0,
                        coloraxis="coloraxis")),
    5, 1
)

fig.add_trace(
    go.Bar(y=def_par,
            x=def_name,
            marker=dict(color=def_par,
                        cmid=0,
                        coloraxis="coloraxis")),
    5, 2
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Top/Bottom PAR per week among rostered players",
                  showlegend=False,
                  height=2000, width=1500
                  )

fig.show()


As we can see above, we have the overperformers within position groups on the left and underperformers within position groups on the right. There is some overlap in some groups if 20 total players at that position are not rostered across all of our teams. I put the kickers and defenses on a single list since you should realistically only be rostering one of each(if you are rostering more, I will make fun of you). I don't put a ton of credence into the underperformer numbers since a lot of those players were hurt, but to answer your question Tom, Brandon Aiyuk has a -5.35 points above replacement value!


In [278]:
fig = go.Figure()

total_par = {NAME_PAIRINGS[roster["owner_id"]]: sum(roster["PAR_values"]) for roster in rosters}
fig.add_trace(
    go.Bar(x=list(total_par.keys()), y=list(total_par.values()),
           marker=dict(color=list(total_par.values()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Aggregate Total Points above Replacement",
                  showlegend=False,
                  xaxis={'categoryorder': 'total descending'},
                  )

In [279]:
fig = go.Figure()

total_par = {NAME_PAIRINGS[roster["owner_id"]]: sum(
    roster["PAR_values"])/CURRENT_WEEK for roster in rosters}
fig.add_trace(
    go.Bar(x=list(total_par.keys()), y=list(total_par.values()),
           marker=dict(color=list(total_par.values()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Aggregate Total Points above Replacement per Week",
                  showlegend=False,
                  xaxis={'categoryorder': 'total descending'},
                  )


## Concluding our Points Above Replacement Analysis

I was actually surprised that a team of replacement players was expected to score 92 points. I think the key is that the variance in these players is also a lot higher than what we expect every week from players on our roster and I'm taking a weekly average, which smooths over some of that added variance. We can also see above that Stian has the highest aggregate points above replacement in the league. Stephane is middle of the pack, but a lot of those come from the two players he just acquired. You should also note that calculating the PAR of defenses is pretty sketchy since most of us just drop and add defenses based on matchups. I left it in regardless since there are a few top tier defenses (Buffalo, and New Orleans for instance) that do genuinely improve team outcomes. I'm going to be interested to see how the points above average stat compares to this. It will probably be similar but I think that it will be able to better rate the typical starting lineup of a given team. 

## Generating Player PAA Estimates

Generating PAA estimates will be a little more difficult than PAR estimates. To simplify things a little bit I will just assume that a team's highest scoring RB is their RB1, second highest is RB2, etc. For the FLEX positions, I will just use the two highest scoring players not already being used as a RB or WR. After I generate these tiers, I will compare a teams' RB1 to an average top 10 RB, their RB2 to an average 11-20 RB, and so on for each position. FLEX positions will be the 20 highest scoring players not being used as a RB 1-20 or WR 1-20. This score should better reflect a teams' positional strength and how well they would do against a team made up of typical starting level players. If a team has a higher aggregate WR PAA score for instance, it can expect more points than the average team from its WRs.

In [280]:
# I'm just going to loop through the rostered players to find the top 10 at each position under the assumption
# that the best/most desirable average players are all rostered

top_average_rostered = {
    "QB" : [], 
    "RB" : [], 
    "WR" : [], 
    "TE" : [], 
    "FLEX" : [], 
    "DEF" : [], 
    "K" : []
}

for player in rostered_players:
    position = rostered_players[player]["position"]
    # if its one of these positions we have to deal with the flex position
    if position in ["RB", "WR", "TE"]:
        # tight ends have only a TE1, whereas RB and WR have RB1/2
        if position == "TE":
            lim_max = 10
        else:
            lim_max = 20

        # if we don't have enough top players, we just use this one
        # as a top player. Otherwise we check points
        if len(top_average_rostered[position]) < lim_max:
            top_average_rostered[position].append(
                rostered_players[player]
            )
        else:
            # get the index of the lowest scoring player
            p_points = rostered_players[player]["points"]
            t_points = [top_player["points"] for top_player in top_average_rostered[position]]
            idx_min = t_points.index(min(t_points))
            # we have to move the player we remove to the FLEX category
            # since by definition they will have a higher points value
            # than players in that list
            if p_points > min(t_points):
                booted_player = top_average_rostered[position][idx_min]
                p_points = booted_player["points"]
                top_average_rostered[position][idx_min] = rostered_players[player]
            else:
                booted_player = rostered_players[player]

            if len(top_average_rostered["FLEX"]) < 20:
                top_average_rostered["FLEX"].append(
                    booted_player
                )
            else:
                t_points = [top_player["points"] for top_player in top_average_rostered["FLEX"]]
                idx_min = t_points.index(min(t_points))
                if p_points > min(t_points):
                    top_average_rostered["FLEX"][idx_min] = booted_player

    else:
        if len(top_average_rostered[position]) < 10:
            top_average_rostered[position].append(
                rostered_players[player]
            )
        else:
            p_points = rostered_players[player]["points"]
            t_points = [top_player["points"] for top_player in top_average_rostered[position]]
            idx_min = t_points.index(min(t_points))
            if p_points > min(t_points):
                top_average_rostered[position][idx_min] = rostered_players[player]

# now that we have the best players at each position, we
# should segment the tiered position groups by scoring
# and then sort the other position groups

for position in FANTASY_POSITIONS:
    if position in ["RB", "WR", "FLEX"]:
        top_players = top_average_rostered[position]
        top_players_sorted = sorted(top_players, key=lambda d: d['points'], reverse=True)
        tier_1 = top_players_sorted[:10]
        tier_2 = top_players_sorted[10:]
        top_average_rostered[f"{position}1"] = tier_1
        top_average_rostered[f"{position}2"] = tier_2
        top_average_rostered.pop(position)
    else:
        top_average_rostered[position] =  sorted(top_average_rostered[position], key=lambda d: d['points'], reverse=True)

top_average_rostered

# now that we have our tiers, lets calculate an average score and weekly score for these tiers

average_scores = {}
for position in top_average_rostered:
    total_score = sum([player["points"] for player in top_average_rostered[position]]) / len(top_average_rostered[position])
    total_weekly_score = total_score / CURRENT_WEEK
    average_scores[position] = {"total": total_score, "weekly": total_weekly_score}

TIERS = ["QB", "RB1", "RB2", "WR1", "WR2", "TE", "FLEX1", "FLEX2", "DEF", "K"]

fig = go.Figure()

average_scores = {k: average_scores[k] for k in TIERS}
weekly_averages = [average_scores[position]["weekly"] for position in average_scores]

fig.add_trace(
    go.Bar(x=list(average_scores.keys()), y=weekly_averages,
           marker=dict(color=weekly_averages, coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Average Player Weekly Score",
                  showlegend=False,
                  )

fig.show()


Cool, now we have generated an expected score for each position location in a players' starting lineup! Notice that although WR1's and RB1's score about the same, the difference between an average RB2 and WR2 is about 1 points. Between the starting positions and the FLEX positions, there is another dropoff of about a point. As expected, the average defense and kicker adds the least amount of expected points to a teams' weekly score. I think the most interesting thing here though is that an average TE is worth about the same as the second tier FLEX, which shows why TE's like Waller, Kelce, etc. are valued so highly. Now that we have this information, we can calculate the points above/below this average value for each of the players on a team. Although it is a bit crude, I will simply assume a teams' top scoring running back is its RB1, its second top scoring RB is its RB2, and so on into the FLEX positions.

In [281]:
for roster in rosters:
    user_players = roster["players"]
    user_players = [rostered_players[player] for player in user_players]
    user_players_sorted = sorted(user_players, key=lambda d: d['points'], reverse=True)
    starters = [[], [], [], [], [], [], []]
    for player in user_players_sorted:
        p_pos = player["position"]
        if p_pos == "QB" and len(starters[0]) < 1:
            player["PAA"] = player["points"] - average_scores["QB"]["total"]
            player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
            player["PAA_name"] = player["name"] + " (QB1)"
            starters[0].append(player)

        elif p_pos == "RB" and len(starters[1]) < 2:
            if len(starters[1]) < 1:
                player["PAA"] = player["points"] - average_scores["RB1"]["total"]
                player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
                player["PAA_name"] = player["name"] + " (RB1)"
                starters[1].append(player)
            else:
                player["PAA"] = player["points"] - average_scores["RB2"]["total"]
                player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
                player["PAA_name"] = player["name"] + " (RB2)"
                starters[1].append(player)

        elif p_pos == "WR" and len(starters[2]) < 2:
            if len(starters[2]) < 1:
                player["PAA"] = player["points"] - average_scores["WR1"]["total"]
                player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
                player["PAA_name"] = player["name"] + " (WR1)"
                starters[2].append(player)
            else:
                player["PAA"] = player["points"] - average_scores["WR2"]["total"]
                player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
                player["PAA_name"] = player["name"] + " (WR2)"
                starters[2].append(player)

        elif p_pos == "TE" and len(starters[3]) < 1:
            player["PAA"] = player["points"] - average_scores["TE"]["total"]
            player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
            player["PAA_name"] = player["name"] + " (TE1)"
            starters[3].append(player)

        elif p_pos in ["RB", "WR", "TE"] and len(starters[4]) < 2:
            if len(starters[4]) < 1:
                player["PAA"] = player["points"] - average_scores["FLEX1"]["total"]
                player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
                player["PAA_name"] = player["name"] + " (FLEX1)"
                starters[4].append(player)
            else:
                player["PAA"] = player["points"] - average_scores["FLEX2"]["total"]
                player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
                player["PAA_name"] = player["name"] + " (FLEX2)"
                starters[4].append(player)

        elif p_pos == "DEF" and len(starters[5]) < 1:
            player["PAA"] = player["points"] - average_scores["DEF"]["total"]
            player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
            player["PAA_name"] = player["name"] + " (DEF)"
            starters[5].append(player)

        elif p_pos == "K" and len(starters[6]) < 1:
            player["PAA"] = player["points"] - average_scores["K"]["total"]
            player["PAA_weekly"] = player["PAA"] / CURRENT_WEEK
            player["PAA_name"] = player["name"] + " (K)"
            starters[6].append(player)
        else:
            continue
        
    # flatten out the starters list for plotting purposes
    roster["expected_starters"] = [item for sublist in starters for item in sublist]


Now that we have generated the PAA values for each player, we can plot the relative strenghts and weaknesses among teams and players.

In [282]:
fig = make_subplots(rows=5, cols=2, shared_yaxes=True,
                    subplot_titles=list(NAME_PAIRINGS.values()))

for i in range(5):
    for j in range(1, 3):
        current = list(NAME_PAIRINGS.keys())[(i+i+j)-1]
        # hacky cause rosters is out of order, fix later if you want
        for roster in rosters:
            if roster["owner_id"] == current:
                current_roster = roster

        starters = current_roster["expected_starters"]
        paa_values = [player["PAA_weekly"] for player in starters]
        paa_names = [player["PAA_name"] for player in starters]

        fig.add_trace(
            go.Bar(y=paa_values,
                   x=paa_names,
                   marker=dict(color=paa_values,
                               cmid=0,
                               coloraxis="coloraxis")),
            i+1, j
        )

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Points Above Average per Week",
                  showlegend=False,
                  height=2000, width=1600
                  )

fig.show()

We can see these team based points above average comparisons in the plot above. It is no surprise that Nahome has above average players at each of his WR positions and his FLEX positions. We can also see why Noah and Stians teams have been so strong, only Kirk Cousins is dragging Noah down whereas Stian's TE position is his main drag on production. Of course, Derrick Henry won't be much use to Noah for the rest of his year which will probably have a knockdown effect on the rest of his team. We can also visualize the most above average players. Note that these won't necessarily be the top performing players, which is a bit counterintuitive. If a WR is doing pretty good but is the WR1 on a team, they won't have as high of a PAA compared to a WR scoring 1 less point as that teams' WR2.

In [283]:
rostered_average_stars = {}
rostered_average_duds = {}

for position in FANTASY_POSITIONS:
    top_players = []
    bot_players = []

    for roster in rosters:
        for player in roster["expected_starters"]:
            if position == "FLEX":
                continue

            else:
                if position in player["fantasy_positions"]:
                    if len(top_players) < LIMIT:
                        top_players.append(
                            player
                        )
                    else:
                        p_points = player["PAA"]
                        paas = [p["PAA"] for p in top_players]
                        idx = paas.index(min(paas))
                        if p_points > paas[idx]:
                            top_players[idx] = player

                    if len(bot_players) < LIMIT:
                        bot_players.append(
                            player
                        )
                    else:
                        p_points = player["PAA"]
                        paas = [p["PAA"] for p in bot_players]
                        idx = paas.index(max(paas))
                        if p_points < paas[idx]:
                            bot_players[idx] = player

    rostered_average_stars[position] = top_players
    rostered_average_duds[position] = bot_players

names = [
    "Quarterbacks", "Tight Ends",
    "RB Over Performers", "RB Under Performers",
    "WR Over Performers", "WR Under Performers",
    "Kickers", "Defenses"
]

fig = make_subplots(rows=4, cols=2, shared_yaxes=True,
                    subplot_titles=names)

for i in range(4):
    current = list(rostered_average_stars.keys())[i]
    star_paa = [player["PAA"] /
                CURRENT_WEEK for player in rostered_average_stars[current]]
    star_name = [player["name"]
                 for player in rostered_average_stars[current]]
    dud_paa = [player["PAA"] /
               CURRENT_WEEK for player in rostered_average_duds[current]]
    dud_name = [player["name"]
                for player in rostered_average_duds[current]]

    if i == 3:
        fig.add_trace(
            go.Bar(y=star_paa,
                x=star_name,
                marker=dict(color=star_paa,
                            cmid=0,
                            coloraxis="coloraxis")),
            1, 2
        )
    else:
        fig.add_trace(
            go.Bar(y=star_paa,
                x=star_name,
                marker=dict(color=star_paa,
                            cmid=0,
                            coloraxis="coloraxis")),
            i+1, 1
        )

    if i == 0 or i == 3:
        continue

    fig.add_trace(
        go.Bar(y=dud_paa,
               x=dud_name,
               marker=dict(color=dud_paa,
                           cmid=0,
                           coloraxis="coloraxis")),
        i+1, 2
    )

k_paa = [player["PAA"] /
         CURRENT_WEEK for player in rostered_average_stars["K"]]
k_name = [player["name"]
          for player in rostered_average_stars["K"]]
def_paa = [player["PAA"] /
           CURRENT_WEEK for player in rostered_average_duds["DEF"]]
def_name = [player["name"]
            for player in rostered_average_duds["DEF"]]

fig.add_trace(
    go.Bar(y=k_paa,
           x=k_name,
           marker=dict(color=k_paa,
                       cmid=0,
                       coloraxis="coloraxis")),
    4, 1
)

fig.add_trace(
    go.Bar(y=def_paa,
           x=def_name,
           marker=dict(color=def_paa,
                       cmid=0,
                       coloraxis="coloraxis")),
    4, 2
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Top/Bottom PAA per week Among Team Starters",
                  showlegend=False,
                  height=2000, width=1500
                  )

fig.show()


Above we can see the average starting points above/below expectation for each teams' expected starting lineup. Since there are only 10 starting QBs, TEs, DEFs, and Ks, we can just roll those plots into one. Note that what I was talking about with Nahome in the WR overperformance plot. He has Cooper Kupp, DJ Moore, and Stefon Diggs even though one starts in his FLEX. Note this player is Diggs, who has a higher PAA than Ja'Marr Chase despite a lower total points value on the season. This is the counterintutive nature of this stat that I am talking about. Since it compares players to others playing in that same position, Nahome has the benefit of always having a FLEX that is expected to perform better than the other teams' flex player. In this way, PAA is a better measure of team strength relative to other teams than PAR was. (If this part doesn't make sense, text the group chat about it!) 

For the last two plots, we can measure the aggregate PAA for a team compared to the other teams to get a sense of team strength. 

In [284]:
fig = go.Figure()

total_paa = {NAME_PAIRINGS[roster["owner_id"]]: sum([player["PAA"] for player in roster["expected_starters"]]) for roster in rosters}
fig.add_trace(
    go.Bar(x=list(total_paa.keys()), y=list(total_paa.values()),
           marker=dict(color=list(total_paa.values()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Aggregate Total Points above Average",
                  showlegend=False,
                  xaxis={'categoryorder': 'total descending'},
                  )


This makes a lot of sense! Whereas most everyone had a team with positive points above replacement, we see that only the stronger teams in the league have high aggregate PAA scores.

In [285]:
fig = go.Figure()

weekly_paa = {NAME_PAIRINGS[roster["owner_id"]]: sum([player["PAA_weekly"] for player in roster["expected_starters"]]) for roster in rosters}
fig.add_trace(
    go.Bar(x=list(weekly_paa.keys()), y=list(weekly_paa.values()),
           marker=dict(color=list(weekly_paa.values()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn),
                  title="Weekly Total Points above Average",
                  showlegend=False,
                  xaxis={'categoryorder': 'total descending'},
                  )


We can see above that Noah (boosted by a now injured Henry) had expected starters who would score 10 more points than an average team). At the other end of the spectrum, Ben has a -13 aggregate weekly points above average after trading away his top players.

## But What about Aiyuk?

That is true Tom, don't worry. Even though Aiyuk isn't a starter he has a special place in our hearts so I will calculate his points below average based on every position we could start him in.

In [287]:
average_scores

rostered_players["6803"]

print(f"If Aiyuk was starting as your WR1, he would have a weekly PAA of {(rostered_players['6803']['points'] - average_scores['WR1']['total']) / CURRENT_WEEK}")
print(f"If Aiyuk was starting as your WR2, he would have a weekly PAA of {(rostered_players['6803']['points'] - average_scores['WR2']['total']) / CURRENT_WEEK}")
print(f"If Aiyuk was starting as your FLEX1, he would have a weekly PAA of {(rostered_players['6803']['points'] - average_scores['FLEX1']['total']) / CURRENT_WEEK}")
print(f"If Aiyuk was starting as your FLEX2, he would have a weekly PAA of {(rostered_players['6803']['points'] - average_scores['FLEX2']['total']) / CURRENT_WEEK}")

If Aiyuk was starting as your WR1, he would have a weekly PAA of -12.528749999999997
If Aiyuk was starting as your WR2, he would have a weekly PAA of -9.10875
If Aiyuk was starting as your FLEX1, he would have a weekly PAA of -6.98175
If Aiyuk was starting as your FLEX2, he would have a weekly PAA of -5.486749999999999


We can in fact conclude that Aiyuk is ASS.